In [1]:
!cat .devcontainer/devcontainer.json

'cat' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [2]:
!cat .devcontainer/docker-compose.yaml

'cat' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [3]:
!cat .devcontainer/Dockerfile

'cat' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [4]:
!pyspark

Missing Python executable 'python3', defaulting to 'C:\Users\21037\AppData\Local\Programs\Python\Python313\Scripts\..' for SPARK_HOME environment variable. Please install Python or specify the correct Python executable in PYSPARK_DRIVER_PYTHON or PYSPARK_PYTHON environment variable to detect SPARK_HOME safely.


ϵͳ�Ҳ���ָ����·����
ϵͳ�Ҳ���ָ����·����


In [ ]:
from pyspark.sql import SparkSession
# 创建了一个 SparkSession 对象。SparkSession 是与 Spark 进行交互的入口
spark = SparkSession.builder.master("local").getOrCreate()
# SparkContext 是 Spark 的核心，用于连接到 Spark 集群并负责与集群管理器进行通信，可以用来执行分布式任务
sc = spark.sparkContext


In [ ]:
# 从指定的文件创建一个 RDD。
# PySpark 假设我们引用的是 HDFS 上的文件。
data = sc.textFile('demo_file.txt')
print(type(data))
# collect() 会将 RDD 中的元素收集到一个列表中。
data.collect()

In [ ]:
# PySpark 会跟踪原始数据所在的位置。
# MapPartitionsRDD 类似于我们创建的所有 RDD 的一个数组
data

In [ ]:
# Simple MapReduce task: Summations
data = sc.textFile('number.txt')
data.collect()

In [ ]:
# 在这里使用strip()是多余的, 这只是为了展示一个例子
intdata = data.map(lambda n: int(n))
intdata.collect()

In [ ]:
intdata.reduce(lambda x,y: x+y)

In [ ]:
# RDD.map()
def polynomialk(x):
return x**2 + 1
data = sc.textFile('number.txt')
data.collect()
doubles = data.map(lambda n: int(n)).map(lambda n: 2*n)
doubles.collect()
data.map(lambda n: int(n)).map(polynomialk).collect()

In [ ]:
## RDD.filter()
data = sc.textFile('number.txt').map(lambda n: int(n))
evens = data.filter(lambda n: n%2==0)
print(evens.collect())
odds = data.filter(lambda n: n%2!=0)
print(odds.collect())
sc.addPyFile('prime.py')
from prime import is_prime
# filter()接受一个布尔函数作为参数，只保留评估为True的元素。
primes = data.filter(is_prime)
print(primes.collect())

In [ ]:
# RDD.sample()
# sample(withReplacement, fraction, [seed])
# RDD.sample()主要用于在数据的一小部分上进行测试。
data = sc.textFile('number.txt').map(lambda n: int(n))
samp = data.sample(False, 0.5)
print(samp.collect())
samp = data.sample(True, 0.5)
print(samp.collect())

In [ ]:
# 如果我的RDD元素比数字更复杂怎么办？
## 类似数据库的文件
data = sc.textFile('scientists.txt')
data.collect()

In [ ]:
# 最初读取时，每一行都是RDD中的一个单独元素。
# 在每个元素上分割空格后，我们得到了我们想要的——每个元素都是一个字符串元组。
# 注意：RDD.collect()返回一个列表，但在RDD内部，元素是元组，而不是列表。
data = data.map(lambda line: line.split())
data.collect()

In [ ]:
## RDD.distinct()
data = sc.textFile('scientists.txt')
data = data.map(lambda line: line.split())
fields = data.map(lambda t: t[3]).distinct()
fields.collect()

In [ ]:
## RDD.flatMap()
data = sc.textFile('numbers_weird.txt')
data.collect()
#同一个数字列表，但现在它们不是每行一个了...
#来自PySpark文档：flatMap(func)类似于map，但是每个输入项可以映射到0个或更多的输出项（所以func应该返回一个序列而不是单个项）。 https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

In [ ]:
## RDD.flatMap()
# 所以我们可以认为flatMap()为每个RDD元素产生一个列表，然后将这些列表连接起来。但至关重要的是，输出是另一个RDD，而不是列表。这种操作称为展平，它是函数式编程中的一个常见模式。
flattened = data.flatMap(lambda line: [x for x in line.split()])
flattened.collect()
flattened.map(lambda n: int(n)).reduce(lambda x,y: x+y)

In [ ]:
# RDD.count()
data = sc.textFile('demo_file.txt')
data = data.flatMap(lambda line: line.split())
data = data.map(lambda w: w.lower())
data.collect()

In [ ]:
uniqwords = data.distinct()
uniqwords.count()

In [ ]:
# RDD.countByKey()
# 注意：在上面的例子中，每个单词都有一个键0，但请注意，在countByKey产生的字典中，值对应于该键出现了多少次。这是因为countByKey统计每个键出现的次数并忽略它们的值。
data = sc.textFile('demo_file.txt')
data = data.flatMap(lambda line: line.split())
data = data.map(lambda w: (w.lower(), 0))
data.countByKey()

In [ ]:
!cat ps_wordcount.py

In [ ]:
!pwd
!spark-submit ps_wordcount.py demo_file.txt wc_demo

In [ ]:
!cat wc_demo/*